In [35]:
# installing altair_nx
!pip install altair_nx

In [2]:
# Necessary libraries
import networkx as nx
import pandas as pd
import altair as alt
import altair_nx as anx
print(alt.__version__)

5.4.1


In [48]:
# Upload edges and nodes
from google.colab import files
uploaded = files.upload()

# read edges and nodes
edges_df = pd.read_csv('CA_links.csv')
nodes_df = pd.read_csv('CA_prices.csv')

In [37]:
# convert ids to string
nodes_df['Node ID'] = nodes_df['Node ID'].astype(str)
edges_df = edges_df.astype({'Start Node ID': 'str', 'End Node ID': 'str'})

# check the type
type(nodes_df['Node ID'].iloc[0])

str

In [38]:
# prune the edges with no valid node ids
for i,j in zip(edges_df['Start Node ID'].values,edges_df['End Node ID'].values):
  if ((i not in nodes_df['Node ID'].values) or (j not in nodes_df['Node ID'].values)):
    edges_df.drop(edges_df[(edges_df['Start Node ID']==i) & (edges_df['End Node ID']==j)].index[0], inplace=True)

In [39]:
# add a fake node for coordinate adjustment
nodes_df = pd.concat([nodes_df, pd.DataFrame([[2021, 5, 15,	'1', 'no_name', 'CA',	115.0, 0, 0, -124.6, 42]], columns=nodes_df.columns)], ignore_index=True)
nodes_df.tail(2)

,Year,Month,Day,Node ID,Node Name,Zone,Nominal Voltage (kV),Average LMP ($/MWh) - Peak,Average LMP ($/MWh) - Offpeak,Longitude,Latitude
5245,2021,5,15,1134902,TNZOND 2,CA_PGAE,60.0,24.98,35.03,-121.7,37.8
5246,2021,5,15,1,no_name,CA,115.0,0.00,0.00,-124.6,42.0


In [40]:
# Create edges and nodes
nodes = [val for val in nodes_df['Node ID']]
edges = [(val1,val2) for val1,val2 in zip(edges_df['Start Node ID'],edges_df['End Node ID'])]

# Create an empty graph
G = nx.Graph()

# Add nodes
# G.add_node(1)
G.add_nodes_from(nodes)

# Add edges
# G.add_edge(1, 2)
G.add_edges_from(edges)

In [44]:
# Assign the coordinates
pos = {i:tuple(nodes_df[['Longitude','Latitude']][nodes_df['Node ID']==i].values[0]) for i in nodes}

# Add all attributes to graph
nodes_dic = {i:i for i in nodes}
price = {i:nodes_df['Average LMP ($/MWh) - Offpeak'][nodes_df['Node ID']==i].iloc[0] for i in nodes}
nx.set_node_attributes(G, nodes_dic, 'node_id')
nx.set_node_attributes(G, price, 'price_heat')

# As with other altair charts, you must sometimes disable max rows
alt.data_transformers.disable_max_rows()
viz = anx.draw_networkx(G,
                        pos = pos,
                        node_colour='price_heat',
                        node_cmap='viridis',
                        node_size=50,
                        node_tooltip=['node_id','price_heat'],
                        # chart_width = 350,
                        # chart_height = 288,
                        chart_padding = 0.00
                       ).properties(title = 'CAISO Heatmap')

In [46]:
# Upload TopoJSON of CA counties
# from google.colab import files
# uploaded = files.upload()

# read TopoJSON of CA counties
import json

# Open and read the JSON file
with open('caCountiesTopoSimple.json', 'r') as file:
    var_topojson = json.load(file)

data_topojson = alt.InlineData(values=var_topojson, format=alt.DataFormat(feature='subunits',type='topojson'))

# chart object
CA_map = alt.Chart(data_topojson).mark_geoshape(filled=False, stroke='black').project(
    type='identity', reflectY=True,
    # type= 'albersUsa',
)

In [47]:
# Visualization
alt.layer(CA_map, viz).properties(width = 550, height = 500)

alt.LayerChart(...)